Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [61]:
import functools
from dataclasses import dataclass 
import numpy as np

from tqdm.auto import tqdm
from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [62]:
UNIVERSE_SIZE = 100
NUM_SETS = 10
DENSITY = 0.2

In [63]:
# DON'T EDIT THESE LINES!

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)


def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper


@counter
def cost(solution):
    """Returns the cost of a solution (to be minimized) tracking number of calls"""
    return COSTS[solution].sum()

# Squillero's greedy solution

## Helper Functions

In [64]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def num_covered(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.sum(np.logical_or.reduce(SETS[solution]))

@dataclass
class Individual:
    genome : np.ndarray
    fitness : float = None

def fitness(Individual):
    """Returns the cost of a solution (to be minimized) tracking number of calls"""
    return int(num_covered(Individual.genome)), -float(cost(Individual.genome))

def parent_selection(population):
    candidates = sorted(np.random.choice(population, 2), key=lambda e: e.fitness , reverse=True)
    return candidates[0]

def xover(p1: Individual, p2: Individual):
    m = np.random.rand(NUM_SETS) < 0.5
    genome = p1.genome.copy()
    genome[m]=p2.genome[m]
    o = Individual(genome)
   # ic(p1, p2, m, o)
    return o

def mutation(p : Individual):
    genome = p.genome.copy()
    x = 0
    while x < 0.1 :
            
        index = np.random.randint(NUM_SETS)
        genome[index] = not genome[index]
        x = np.random.random()
    return Individual(genome)

## Have Fun!

In [65]:
POPULATION_SIZE = 10
population = [Individual(np.random.rand(NUM_SETS) < 0.5) for i in range(POPULATION_SIZE)]
for i in population:
    i.fitness = fitness(i)

OFFSPRING_SIZE = 4 #need steady state approach cause size < population size

MAX_GENERATIONS = 100

for g in range(MAX_GENERATIONS):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        if np.random.random() < .3:
            #mutation
            parent = parent_selection(population)
            o = mutation(parent)
        else :
            #recombination
            i1 = parent_selection(population)
            i2 = parent_selection(population)
            o = xover(i1, i2)
        offspring.append(o)

    for i in offspring:
        i.fitness = fitness(i) #only looking at the fitness of the offspring not of the paretns, static approach

population.extend(offspring)
population.sort(key=lambda e: e.fitness, reverse=True)
population = population[:POPULATION_SIZE]    

In [66]:
ic(mutation(population[0].fitness), cost.calls)

AttributeError: 'tuple' object has no attribute 'genome'